In [13]:
import pandas as pd
from bs4 import BeautifulSoup

In [17]:
#This code snippet is parsing a baby table
#To understand the structure of parsing these tables
html_string = '''
      <table>
            <tr>
                <td> This </td>
                <td> is  </td>
                <td> a  </td>
                <td> row  </td>
            </tr>
        </table>
    '''
    
soup = BeautifulSoup(html_string, 'lxml') # Parse the HTML as a string
    
table = soup.find_all('table')[0] # Grab the first table
    
new_table = pd.DataFrame(columns=range(0,4), index = [0]) # I know the size 
    
row_marker = 0
for row in table.find_all('tr'): #finding all the table rows
    column_marker = 0 # a simple marker
    columns = row.find_all('td') #grabbing table data for each table row
    for column in columns:
        new_table.iat[row_marker,column_marker] = column.get_text()
        #pd.dataframe.iat = Access a single value for a row/column pair by integer position.
        #column.get_text() grabbing text of the table data
        column_marker += 1 #
    
new_table


,0,1,2,3
0,This,is,a,row


In [51]:
import requests
url = "https://www.fantasypros.com/nfl/reports/leaders/qb.php?year=2015"
response = requests.get(url)
#response.text[:100] # Access the HTML with the text property

In [27]:
response.text[:50]

'\r\n<!DOCTYPE html>\n<html lang="en">\n\n<head>\n    <ti'

In [56]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
    
class HTMLTableParser: #Creating an html table parser
       
    def parse_url(self, url): 
        response = requests.get(url) #get URL of what your parsing through
        soup = BeautifulSoup(response.text, 'lxml')
        #if affiliate_item and affiliate_item.has_key('id'):
        return [(table['id'],self.parse_html_table(table))\
                    for table in soup.find_all('table')]  
    
    def parse_html_table(self, table):
        n_columns = 0
        n_rows=0
        column_names = []
    
        # Find number of rows and columns
        # we also find the column titles if we can
        for row in table.find_all('tr'):
                
            # Determine the number of rows in the table
            td_tags = row.find_all('td')
            if len(td_tags) > 0:
                n_rows+=1
                if n_columns == 0:
                    # Set the number of columns for our table
                    n_columns = len(td_tags)
                        
                # Handle column names if we find them
            th_tags = row.find_all('th') 
            if len(th_tags) > 0 and len(column_names) == 0:
                for th in th_tags:
                    column_names.append(th.get_text())
    
            # Safeguard on Column Titles
        if len(column_names) > 0 and len(column_names) != n_columns:
            raise Exception("Column titles do not match the number of columns")
    
        columns = column_names if len(column_names) > 0 else range(0,n_columns)
        df = pd.DataFrame(columns = columns,
                            index= range(0,n_rows))
        row_marker = 0
        for row in table.find_all('tr'):
            column_marker = 0
            columns = row.find_all('td')
            for column in columns:
                df.iat[row_marker,column_marker] = column.get_text()
                column_marker += 1
            if len(columns) > 0:
                row_marker += 1
                    
            # Convert to float if possible
        for col in df:
            try:
                df[col] = df[col].astype(float)
            except ValueError:
                pass
            
        return df

In [60]:
hp = HTMLTableParser()
table = hp.parse_url(url) # Grabbing the table from the tuple
table.head()


AttributeError: 'NoneType' object has no attribute 'head'

In [67]:
table = url
for url in soup.find_all('id'):
    print(url)

In [70]:
soup.find_all('td')

[<td> This </td>, <td> is  </td>, <td> a  </td>, <td> row  </td>]

In [65]:
url

'https://www.fantasypros.com/nfl/reports/leaders/qb.php?year=2015'